# **Deribit Open Interest**
Server version of this (Will publish Deribit open interest data to Redis for strategy consumption): https://github.com/r0bbar/siglab/blob/master/siglab_py/market_data_providers/deribit_options_expiry_provider.py

https://norman-lm-fung.medium.com/monitoring-incoming-deribit-open-interest-fd8c8d596ca0

# Imports

In [1]:
!pip install ccxt
import os
import sys
import traceback
from enum import Enum
import argparse
from datetime import datetime, timedelta
import time
from typing import Dict, Union, Tuple
import json
import asyncio
import logging
from ccxt import deribit
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.9/117.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


# **Parameters**

In [2]:

param : Dict = {
    'market' : 'BTC',

    # Provider ID is part of mds publish topic.
    'provider_id' : 'b0f1b878-c281-43d7-870a-0347f90e6ece',

    'archive_file_name' : "deribit_options_expiry.csv",

    # Publish to message bus
    'mds' : {
        'topics' : {
            'deribit_options_expiry_publish_topic' : 'deribit-options-expiry'
        },
        'redis' : {
            'host' : 'localhost',
            'port' : 6379,
            'db' : 0,
            'ttl_ms' : 1000*60*15 # 15 min?
        }

    }
}

logging.Formatter.converter = time.gmtime
logger = logging.getLogger()
log_level = logging.INFO # DEBUG --> INFO --> WARNING --> ERROR
logger.setLevel(log_level)
format_str = '%(asctime)s %(message)s'
formatter = logging.Formatter(format_str)
sh = logging.StreamHandler()
sh.setLevel(log_level)
sh.setFormatter(formatter)
logger.addHandler(sh)
# fh = logging.FileHandler(f"{param['job_name']}.log")
# fh.setLevel(log_level)
# fh.setFormatter(formatter)
# logger.addHandler(fh)

class LogLevel(Enum):
    CRITICAL = 50
    ERROR = 40
    WARNING = 30
    INFO = 20
    DEBUG = 10
    NOTSET = 0

# **Helper Functions**

In [3]:
def log(message : str, log_level : LogLevel = LogLevel.INFO):
    if log_level.value<LogLevel.WARNING.value:
        logger.info(f"{datetime.now()} {message}")

    elif log_level.value==LogLevel.WARNING.value:
        logger.warning(f"{datetime.now()} {message}")

    elif log_level.value==LogLevel.ERROR.value:
        logger.error(f"{datetime.now()} {message}")

def fetch_ohlcv_one_candle(
    exchange,
    normalized_symbol : str,
    timestamp_ms : int,
    ref_timeframe : str = '1m'
):
    candles = exchange.fetch_ohlcv(symbol=normalized_symbol, since=int(timestamp_ms), timeframe=ref_timeframe, limit=1)
    one_candle = {
            'timestamp_ms' : candles[0][0],
            'open' : candles[0][1],
            'high' : candles[0][2],
            'low' : candles[0][3],
            'close' : candles[0][4],
            'volume' : candles[0][5]
        } if candles and len(candles)>0 else None

    return one_candle

def timestamp_to_datetime_cols(pd_candles : pd.DataFrame):
    pd_candles['datetime'] = pd_candles['timestamp_ms'].apply(
        lambda x: datetime.fromtimestamp(int(x.timestamp()) if isinstance(x, pd.Timestamp) else int(x / 1000))
    )
    pd_candles['datetime'] = pd.to_datetime(pd_candles['datetime'])
    pd_candles['datetime'] = pd_candles['datetime'].dt.tz_localize(None)
    pd_candles['datetime_utc'] = pd_candles['timestamp_ms'].apply(
        lambda x: datetime.fromtimestamp(int(x.timestamp()) if isinstance(x, pd.Timestamp) else int(x / 1000), tz=timezone.utc)
    )

    # This is to make it easy to do grouping with Excel pivot table
    pd_candles['year'] = pd_candles['datetime'].dt.year
    pd_candles['month'] = pd_candles['datetime'].dt.month
    pd_candles['day'] = pd_candles['datetime'].dt.day
    pd_candles['hour'] = pd_candles['datetime'].dt.hour
    pd_candles['minute'] = pd_candles['datetime'].dt.minute
    pd_candles['dayofweek'] = pd_candles['datetime'].dt.dayofweek  # dayofweek: Monday is 0 and Sunday is 6

def fetch_deribit_btc_option_expiries(
    market: str = 'BTC'
) -> list[Tuple[str, float]]:
    exchange = deribit()
    instruments = exchange.public_get_get_instruments({
        'currency': market,
        'kind': 'option',
        # 'expired': 'true'
    })['result']

    index_price = exchange.public_get_get_index_price({
        'index_name': f"{market.lower()}_usd"
    })['result']['index_price']
    index_price = float(index_price)

    expiry_data : Dict[str, float] = {}
    for instrument in instruments:
        expiry_timestamp = int(instrument["expiration_timestamp"]) / 1000
        expiry_date = datetime.utcfromtimestamp(expiry_timestamp)

        ticker = exchange.public_get_ticker({
            'instrument_name': instrument['instrument_name']
        })['result']

        open_interest = ticker.get("open_interest", 0)  # Open interest in BTC
        open_interest = float(open_interest)
        notional_value : float = open_interest * index_price  # Convert to USD

        expiry_str : str = expiry_date.strftime("%Y-%m-%d")
        if expiry_str not in expiry_data:
            expiry_data[expiry_str] = 0
        expiry_data[expiry_str] += notional_value

    sorted_expiry_data = sorted(expiry_data.items())
    return sorted_expiry_data

def _fetch_historical_daily_candle_height(
        exchange,
        normalized_symbol : str,
        timestamp_ms : int,
        offset_days : int,
        candle_height : float,
        reload_candle_height : bool = False
    ):
    if not candle_height or reload_candle_height:
        dt = datetime.fromtimestamp(int(timestamp_ms/1000)) + timedelta(days=offset_days)
        dt = datetime(dt.year, dt.month, dt.day)
        timestamp_ms = int(dt.timestamp()) * 1000
        if dt < datetime(datetime.today().year, datetime.today().month, datetime.today().day):
            historical_day_candle = fetch_ohlcv_one_candle(exchange=exchange, normalized_symbol=normalized_symbol, timestamp_ms=timestamp_ms, ref_timeframe='1d')
            if historical_day_candle:
                return historical_day_candle['close'] - historical_day_candle['open']
            else:
                return None
        else:
            return None
    else:
        return None

Fetch Data

In [4]:
start = time.time()
expiry_data = fetch_deribit_btc_option_expiries(market = param['market'])
elapsed_sec = int((time.time() - start))
log(f"Took {elapsed_sec} sec to fetch option expiry data from Deribit")
pd_expiry_data = pd.DataFrame([ { 'datetime' : x[0], 'notional_usd' : x[1] } for x in expiry_data ])
pd_expiry_data['notional_usd'] = pd_expiry_data['notional_usd'].apply(lambda x: f"{x:,.2f}")
pd_expiry_data

INFO:root:2025-01-02 03:24:46.581600 Took 78 sec to fetch option expiry data from Deribit
2025-01-02 03:24:46,581 2025-01-02 03:24:46.581600 Took 78 sec to fetch option expiry data from Deribit


,datetime,notional_usd
0,2025-01-02,"140,647,864.82"
1,2025-01-03,"1,852,985,533.46"
2,2025-01-04,"39,375,325.42"
3,2025-01-10,"944,751,447.89"
4,2025-01-17,"232,378,035.76"
5,2025-01-31,"5,629,864,468.78"
6,2025-02-28,"1,185,722,362.72"
7,2025-03-28,"7,302,295,097.94"
8,2025-06-27,"2,553,243,460.86"
9,2025-09-26,"765,022,599.37"
